# Import Libraries

In [22]:

import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

## Import MNIST

In [3]:
def sort_by_target(mnist):
    reorder_train = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[:60000])]))[:, 1]
    reorder_test = np.array(sorted([(target, i) for i, target in enumerate(mnist.target[60000:])]))[:, 1]
    mnist.data[:60000] = mnist.data[reorder_train]
    mnist.target[:60000] = mnist.target[reorder_train]
    mnist.data[60000:] = mnist.data[reorder_test + 60000]
    mnist.target[60000:] = mnist.target[reorder_test + 60000]
    
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [4]:
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True)
    mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings
    sort_by_target(mnist) # fetch_openml() returns an unsorted dataset
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')
mnist["data"], mnist["target"]

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([0, 0, 0, ..., 9, 9, 9], dtype=int8))

## Split data
#### Stratified y to avoid over-representing a single digit.

In [11]:
X, y = mnist["data"], mnist["target"]
X_train, X_test, y_train, y_test = X[:60000],X[60000:],y[:60000],y[60000:]
train_test_split(X_train,y_train,test_size=0.2, random_state=42, stratify=y_train)

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([1, 3, 6, ..., 5, 1, 2], dtype=int8), array([7, 7, 3, ..., 8, 6, 7], dtype=int8)]

## Create Pipeline and Train

In [15]:
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel='poly',degree=3, coef0=1, C=5, gamma='auto'))
])
svm_clf.fit(X,y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=5, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

## Determine model accuracy

In [21]:
cross_val_score(svm_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.97270546, 0.9700985 , 0.97149572])

In [23]:
y_pred = svm_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.9992666666666666

In [24]:
confusion_matrix(y_train, y_pred)

array([[5923,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0, 6738,    0,    0,    1,    0,    0,    3,    0,    0],
       [   0,    1, 5957,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0, 6126,    0,    2,    0,    1,    1,    1],
       [   0,    0,    0,    0, 5839,    0,    0,    0,    0,    3],
       [   0,    0,    0,    2,    0, 5419,    0,    0,    0,    0],
       [   0,    0,    0,    0,    2,    0, 5916,    0,    0,    0],
       [   1,    4,    0,    0,    0,    0,    0, 6254,    0,    6],
       [   0,    2,    0,    0,    0,    0,    0,    0, 5848,    1],
       [   0,    0,    0,    0,    5,    0,    0,    8,    0, 5936]],
      dtype=int64)